# Vehicle Type Classification

K. Pyong-Kun, L. Kil-Taek (2017). Vehicle Type Classification Using Bagging and Convolutional Neural Network on Multi View Surveillance Image. Retrieved from https://openaccess.thecvf.com/content_cvpr_2017_workshops/w9/papers/Kim_Vehicle_Type_Classification_CVPR_2017_paper.pdf
<br>
<br>
Dataset @ http://podoce.dinf.usherbrooke.ca/challenge/dataset/
